In [3]:
import requests
import pandas as pd
import time

# Set API Key dan Author ID Scopus
api_key = "81b43f2badb4f475df7a6c3d6761e9d5"
author_id = "57190939346"

# URL dasar untuk Scopus Search API
base_url = "https://api.elsevier.com/content/search/scopus"

# Parameter paginasi
count = 25
start = 0
all_publications = []

# Looping ambil data
while True:
    params = {
        "query": f"AU-ID({author_id})",
        "count": count,
        "start": start,
        "field": "dc:title,prism:publicationName,prism:coverDate,eid,citedby-count"
    }

    headers = {
        "X-ELS-APIKey": api_key,
        "Accept": "application/json"
    }

    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code != 200:
        print(f"Error: {response.status_code}\n{response.text}")
        break

    data = response.json()
    entries = data.get("search-results", {}).get("entry", [])

    if not entries:
        break

    df = pd.json_normalize(entries)
    all_publications.append(df)

    print(f"✅ Retrieved {len(entries)} entries. Total so far: {len(all_publications)}")

    total_results = int(data["search-results"].get("opensearch:totalResults", 0))
    if start + count >= total_results:
        break

    start += count
    time.sleep(1)

# Gabungkan semua DataFrame
pub_df_raw = pd.concat(all_publications, ignore_index=True)

# Bersihkan dan transformasi kolom
pub_df = pd.DataFrame({
    "Title": pub_df_raw.get("dc:title", pub_df_raw.get("prism:title")),
    "Journal": pub_df_raw.get("prism:publicationName"),
    "Publication_Date": pub_df_raw.get("prism:coverDate"),
    "Citations": pd.to_numeric(pub_df_raw.get("citedby-count", 0), errors="coerce").fillna(0).astype(int),
    "EID": pub_df_raw.get("eid")
})

pub_df["Tahun"] = pub_df["Publication_Date"].str[:4]
pub_df.insert(0, "No", range(1, len(pub_df) + 1))

# Tampilkan hasil
print("\n📄 Data publikasi yang berhasil diproses:")
print(pub_df.head(10))

# Simpan ke Excel (jika ingin)
pub_df.to_excel("scopus_result.xlsx", index=False)


✅ Retrieved 25 entries. Total so far: 1
✅ Retrieved 23 entries. Total so far: 2

📄 Data publikasi yang berhasil diproses:
   No                                              Title  \
0   1  MODELING THE INCIDENCE OF MALNUTRITION IN BOGO...   
1   2  Analyzing multilevel model of educational data...   
2   3  SIMULATION STUDY OF HIERARCHICAL BAYESIAN APPR...   
3   4  A PRELIMINARY STUDY OF SENTIMENT ANALYSIS ON C...   
4   5  TRANSFER FUNCTION AND ARIMA MODEL FOR FORECAST...   
5   6  MTSClust with Handling Missing Data Using VAR-...   
6   7  POISSON-LOGNORMAL MODEL WITH MEASUREMENT ERROR...   
7   8  Vector Autoregressive-Moving Average Imputatio...   
8   9  Simulation for Time Series Classification Usin...   
9  10  Small Area Estimation of Internet Users Propor...   

                                             Journal Publication_Date  \
0                                           Barekeng       2024-06-01   
1                   Journal on Mathematics Education       2024-02-01  